## Assignment 5: Advanced pandas
Let's continue to use the dataset from the wine magazine used in notebook 'Colab_Lec06_advanced-pandas' to practice data transformation, grouping, and sorting.

<img src="https://secure.static.meredith.com/crt/store/covers/magazines/nmo/9826_l.jpg">

In [2]:
csvurl="https://gist.githubusercontent.com/clairehq/" + \
        "79acab35be50eaf1c383948ed3fd1129/raw/407a02139ae1e134992b90b4b2b8c329b3d73a6a/winemag-data-130k-v2.csv"
import pandas as pd
wine = pd.read_csv(csvurl)
wine.drop(wine.columns[0], axis="columns", inplace=True)
wine.head()

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


#### Question 1: ####  
What is the mean of the points column?

In [3]:
wine.points.mean()

88.43403716087269

#### Question 2: ####  
How many countries are present in this dataset? (Only count each country once)

In [4]:
wine.country.nunique()

41

#### Question 3: ####
How many times does each country appeared in this dataset? Show each country and the corresponding count (show counts in ascending order)

In [5]:
wine.groupby('country').country.value_counts().to_frame().sort_values(by = 'count')

,count
country,
Armenia,1
Bosnia and Herzegovina,1
Slovakia,1
Luxembourg,4
India,4
Switzerland,4
Ukraine,5
Cyprus,6
Macedonia,6


#### Question 4: ####
Create a variable `adjusted_price` containing the adjusted price which is the price subtracted by the average price. *This is called **"centering" transformation** - a method commonly used in the preprocessing step before applying various machine learning algorithms.*

In [6]:
adjusted_price = wine.price - wine.price.mean()
adjusted_price

,price
0,NaN
1,-20.232932
2,-21.232932
3,-22.232932
4,29.767068
...,...
65494,9.767068
65495,-13.232932
65496,-15.232932
65497,-4.232932


#### Question 5: ####
What is the title of the wine that has the highest points-to-price ratio in the dataset?

In [7]:
p2p_ratio = wine.title.to_frame()
p2p_ratio['points-to-price ratio'] = (wine.points / wine.price)
p2p_ratio.sort_values(by="points-to-price ratio", ascending=False).head(1)

,title,points-to-price ratio
64590,Bandit NV Merlot (California),21.5


#### Question 6: ####
Create a series `flavor_counts` that contains two values: the number of wines that has the word "tart" in the `description` column and the number of wines that has the word "berries" in the `description` column. The index of the Series should be "Tart" and "Berries" for the corresponding values.

In [8]:
num_tart = 0
num_berries = 0
for description in (" " + wine.description.str.lower() + " "):
  num_tart += description.count(" tart ")
  num_berries += description.count(" berries ")
nums = [num_tart, num_berries]

flavor_counts = pd.Series(nums)
flavor_counts.index = ['Tart', 'Berries']
flavor_counts

,0
Tart,2415
Berries,628


#### Question 7: ####
Let's convert the points into simple star ratings. A score of 90 or higher counts as 3 stars, a score of at least 80 but less than 90 is 2 stars. Any other score is 1 star.

Also, any wines from France should automatically get 3 stars, regardless of points.

Add this new column `star_ratings` to the dataframe with the number of stars for each wine in the dataset.

In [9]:
def get_stars(row):
  if row.country == 'France':
    return 3
  elif row.points >= 90:
    return 3
  elif 80 <= row.points < 90:
    return 2
  else:
    return 1

wine['star_ratings'] = wine.apply(get_stars, axis='columns')
wine

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery,star_ratings
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia,2
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,2
2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm,2
3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian,2
4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65494,France,Made from young vines from the Vaulorent porti...,Fourchaume Premier Cru,90,45.0,Burgundy,Chablis,NaN,Roger Voss,@vossroger,William Fèvre 2005 Fourchaume Premier Cru (Ch...,Chardonnay,William Fèvre,3
65495,Australia,"This is a big, fat, almost sweet-tasting Caber...",NaN,90,22.0,South Australia,McLaren Vale,NaN,Joe Czerwinski,@JoeCz,Tapestry 2005 Cabernet Sauvignon (McLaren Vale),Cabernet Sauvignon,Tapestry,3
65496,US,"Much improved over the unripe 2005, Fritz's 20...",Estate,90,20.0,California,Dry Creek Valley,Sonoma,NaN,NaN,Fritz 2006 Estate Sauvignon Blanc (Dry Creek V...,Sauvignon Blanc,Fritz,3
65497,US,This wine wears its 15.8% alcohol better than ...,Block 24,90,31.0,California,Napa Valley,Napa,NaN,NaN,Hendry 2004 Block 24 Primitivo (Napa Valley),Primitivo,Hendry,3


#### Question 8: ####
Who are the most common wine reviewers in the dataset? Create a Series whose index is the taster_twitter_handle category from the dataset, and whose values count how many reviews each person wrote.

In [10]:
reviewers = wine.taster_twitter_handle.dropna().unique()

counts = []
for reviewer in reviewers:
  i = 0
  for name in wine.taster_twitter_handle:
    if name == reviewer:
      i += 1
  counts.append(i)

wine_reviewers = pd.Series(counts, index = reviewers).rename("Counts").sort_values(ascending = False)
wine_reviewers

,Counts
@vossroger,13045
@wineschach,7752
@kerinokeefe,5313
@paulgwine,4851
@vboone,4696
@mattkettmann,3035
@JoeCz,2605
@wawinereport,2358
@gordone_cellars,2032
@AnneInVino,1769


#### Question 9: ####
What combination of countries and varieties are most common? Create a Series whose index is a MultiIndexof {country, variety} pairs. For example, a pinot noir produced in the US should map to {"US", "Pinot Noir"}. Sort the values in the Series in descending order based on wine count.

In [11]:
common_combos = wine.groupby(['country', 'variety']).description.agg(['count']).sort_values(by = 'count', ascending = False)
common_combos

count
country variety                        
US      Pinot Noir                 4918
        Cabernet Sauvignon         3649
        Chardonnay                 3412
France  Bordeaux-style Red Blend   2380
Italy   Red Blend                  1870
...                                 ...
        Torbato                       1
        Vespaiolo                     1
        Vespolina                     1
        Vitovska                      1
Uruguay Tempranillo-Tannat            1

[1304 rows x 1 columns]

#### Question 10 #####
Create a Series whose index is reviewers and whose values is the average score given out by that reviewer. Hint: you will need the `taster_name` and `points` columns.

In [98]:
average_scores = wine.groupby('taster_name')['points'].mean()
average_scores

,points
taster_name,
Alexander Peartree,86.014286
Anna Lee C. Iijima,88.380506
Anne Krebiehl MW,90.587903
Carrie Dykes,86.644444
Christina Pickard,89.500000
Fiona Adams,87.090909
Jeff Jenssen,88.273504
Jim Gordon,88.604331
Joe Czerwinski,88.519770
